In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py --no-check-certificate

--2025-06-03 10:48:49--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0.01s   

2025-06-03 10:48:50 (386 KB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [2]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [3]:
import minsearch
import json
from mistralai import Mistral
from mistralai.models import UserMessage
import httpx

In [4]:
import os
from dotenv import load_dotenv


In [5]:
with open('documents.json', 'rt') as h_a:
    docs_row = json.load(h_a)

In [ ]:
docs_row

In [7]:
documents = []

for course_dict in docs_row: #Loop through each course
    for doc in course_dict['documents']: #Loop through each document inside that course:
        doc['course']= course_dict['course'] #Add the course name to each document
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
index = minsearch. Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"] # it's a filter acts like select * from t where course = "data-engineering-zoomcamp"
)


In [10]:
q = 'the course has already started, can I still enroll?'


In [11]:
index.fit(documents)

In [35]:
def search(query):
# give importance to question rather than section
    boost = {'question': 3.0, 'section': 0.5}
    results  = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'}, #optional 
        boost_dict=boost,
        num_results=5    
    )
    return results

In [36]:
load_dotenv()  # This loads variables from .env

True

In [37]:
api_key = os.getenv("API_KEY")

In [38]:
client = Mistral(api_key = api_key, client=httpx.Client(verify=False))

In [39]:
q

'the course has already started, can I still enroll?'

In [40]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [41]:
def llm(prompt):
    response = client.chat.complete(
        model = "mistral-large-latest", 
        messages = [UserMessage(content=prompt)]
    )
    return response.choices[0].message.content


In [42]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [43]:
query = 'how do I run kafka?'
rag(query)

'To run Kafka, you can follow the instructions provided in the context for both Java and Python environments.\n\nFor Java Kafka:\n1. Navigate to the project directory.\n2. Run the following command in the terminal:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\nFor Python Kafka:\n1. Create a virtual environment and install the necessary packages.\n   ```\n   python -m venv env\n   source env/bin/activate  # For Windows, use: env/Scripts/activate\n   pip install -r ../requirements.txt\n   ```\n2. Ensure that the virtual environment is activated every time you need to run the Python file.\n   ```\n   source env/bin/activate  # For Windows, use: env/Scripts/activate\n   ```\n3. Make sure all Docker images are up and running before executing the Python file.'

In [44]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. Even if you don't register, you're still eligible to submit the homework."